# Circuit Generation

The python script below creates 3 unary multipliers, i.e. circuits that multiply a single input integer by a fixed prime intenger. In this case we have circuits that multiply input by 13, 7, and 5. The script writes verilog, then uses yosys to convert from an abstract design to a circuit level one, then convert the circuit level design into ANSprolog facts.

In [140]:
%%file circuit-gen.py
import sys, subprocess, json, os
try:
    width = int(sys.argv[1])
except:
    print 'usage: python circuit-gen.py port-width'
    exit()
    
verilog_src = \
"""
module mul13(mul13in, mul13out);
    parameter WIDTH = {0};
    input [WIDTH-1:0] mul13in;
    output [WIDTH-1:0] mul13out;
    
    assign mul13out = mul13in * 13;
endmodule

module mul7(mul7in, mul7out);
    parameter WIDTH = {0};
    input [WIDTH-1:0] mul7in;
    output [WIDTH-1:0] mul7out;
    
    assign mul7out = mul7in * 7;
endmodule

module mul5(mul5in, mul5out);
    parameter WIDTH = {0};
    input [WIDTH-1:0] mul5in;
    output [WIDTH-1:0] mul5out;
    
    assign mul5out = mul5in * 5;
endmodule

module mod(modin1,modin2,modout);
    parameter WIDTH = {0};
    input [WIDTH-1:0] modin1;
    input [WIDTH-1:0] modin2;
    output [WIDTH-1:0] modout;
    assign modout = modin1 % modin2;
endmodule

module mul(mulin1,mulin2,mulout);
    parameter WIDTH = {0};
    input [WIDTH-1:0] mulin1;
    input [WIDTH-1:0] mulin2;
    output [WIDTH-1:0] mulout;
    assign mulout = mulin1 % mulin2;
endmodule

module sub(subin1,subin2,subout);
    parameter WIDTH = {0};
    input [WIDTH-1:0] subin1;
    input [WIDTH-1:0] subin2;
    output [WIDTH-1:0] subout;
    assign subout = subin1 % subin2;
endmodule

""".format(width)

yosys_cmd = \
"""
yosys \
-p "synth; abc -liberty mycells.lib; clean; write_json tmp.json" \
-QT -f verilog tmp.v
"""

outfile = open('tmp.v','w')
outfile.write(verilog_src)
outfile.close()

FNULL = open(os.devnull, 'w')
subprocess.call(yosys_cmd,shell=True,stdout=FNULL)

modules = json.load(open('tmp.json'))['modules']
outfile = open('multiplier.lp','w')
for m in modules:
    outfile.write('device({}).\n'.format(m))
    dev = modules[m]
    count = 0
    for p in dev['ports']:
        port = dev['ports'][p]
        outfile.write('device_port_direction({},{},{},{}).\n'.format(m,p,port['direction'],count))
        count += 1
        outfile.write('device_port_width({},{},{}).\n'.format(m,p,len(port['bits'])))
        for i,b in enumerate(port['bits']):
            outfile.write('device_port_bit_wire({},{},{},{}).\n'.format(m,p,i,b))
    for c in dev['cells'].values():
        con = c['connections']
        if c['type'] == 'AND':
            outfile.write('device_gate({},and_gate,({}, {}, {})).\n'.format(m,con['A'][0],con['B'][0],con['Y'][0]))
        if c['type'] == 'OR':
            outfile.write('device_gate({},or_gate,({}, {}, {})).\n'.format(m,con['A'][0],con['B'][0],con['Y'][0]))
        if c['type'] == 'NOT':
            outfile.write('device_gate({},not_gate,({}, {})).\n'.format(m,con['A'][0],con['Y'][0])) 
outfile.close()
os.remove('tmp.v')
os.remove('tmp.json')

Overwriting circuit-gen.py


Here is an example of the circuit specification rules. These facts use the same predicates as were used in the Grid Embedding assignment.

In [141]:
!python2 circuit-gen.py 8
!head multiplier.lp

device(sub).
device_port_direction(sub,subin2,input,0).
device_port_width(sub,subin2,8).
device_port_bit_wire(sub,subin2,0,10).
device_port_bit_wire(sub,subin2,1,11).
device_port_bit_wire(sub,subin2,2,12).
device_port_bit_wire(sub,subin2,3,13).
device_port_bit_wire(sub,subin2,4,14).
device_port_bit_wire(sub,subin2,5,15).
device_port_bit_wire(sub,subin2,6,16).


The rules below, taken from P2, have been modified so that each signal-state, i.e. the presecence or absence of signal on each wire, is associated with a node and key being hashed. So now instead of having two circuits, we create a separate circuit for each combination of node i.d., hash key, and device type.

In [142]:
%%file circuits.lp
input_port(Port) :- device_port_direction(Device,Port,input,C).

output_port(Port) :- device_port_direction(Device,Port,output,C).

port_width(Port,Size) :-
    device_port_width(Device,Port,Size).

% Define all the gate rules
signal(I,K,Device,Out,1-(1-V1)*(1-V2)) :-
    device_gate(Device,or_gate,(In1,In2,Out)),
    signal(I,K,Device,In1,V1); signal(I,K,Device,In2,V2).

signal(I,K,Device,Out,V1*V2) :-
    device_gate(Device,and_gate,(In1,In2,Out)),
    signal(I,K,Device,In1,V1), signal(I,K,Device,In2,V2).

signal(I,K,Device,Out,1-V1) :-
    device_gate(Device,not_gate,(In,Out)),
    signal(I,K,Device,In,V1).

% get input signal
signal(I,K,Device,Wire,Value) :-
    input_signal(I,K,Port,Bit,Value),
    device_port_bit_wire(Device,Port,Bit,Wire).

% get output signal
output_signal(I,K,Bit,Value) :-
    signal(I,K,Device,Wire,Value),
    output_port(Port),
    device_port_bit_wire(Device,Port,Bit,Wire).

Overwriting circuits.lp


In [143]:
!clingo multiplier.lp circuits.lp

clingo version 5.2.1
Reading from multiplier.lp ...
circuits.lp:23:5-37: info: atom does not occur in any rule head:
  input_signal(I,K,Port,Bit,Value)

Solving...
Answer: 1
device_port_direction(sub,subin2,input,0) device_port_direction(sub,subin1,input,1) device_port_direction(sub,subout,output,2) device_port_direction(mul13,mul13in,input,0) device_port_direction(mul13,mul13out,output,1) device_port_direction(mul,mulout,output,0) device_port_direction(mul,mulin2,input,1) device_port_direction(mul,mulin1,input,2) device_port_direction(mod,modin1,input,0) device_port_direction(mod,modout,output,1) device_port_direction(mod,modin2,input,2) device_port_direction(mul7,mul7out,output,0) device_port_direction(mul7,mul7in,input,1) device_port_direction(mul5,mul5in,input,0) device_port_direction(mul5,mul5out,output,1) input_port(subin2) input_port(subin1) input_port(mul13in) input_port(mulin2) input_port(mulin1) input_port(modin1) input_port(modin2) input_port(mul7in) input_port(mul5in) outpu

In [144]:
%%file ast.lp

bit(0;1).

#const bits=8.
#const max_index = 30.

bit_index(0..bits-1).

id(0..10).

rooted(0).
rooted(B) :- rooted(A), edge(A,B).
:- node(B), not rooted(B).

%%% Each node has 0-2 children and is connected to the root
%%% Childen have greater ids than their parents
0 {edge(A,B):id(B),B>A} 2 :- rooted(A).

%%% Children cant have multiple parents
%%% It might actually be useful to allow this, so not really an AST anymore
%%% :- id(I), not 0 #sum{1,P:edge(P,I)} 1.
          
%%% TODO:
% All subdags contain at least one key item if the head of the tree is an operation
% This is just an optimization and shouldnt affect the final answer (assuming we tell it to find optimal)
%%%

node(I) :- rooted(I).
binary(I) :- node(I), 2{edge(I,C)}2.
unary(I) :- node(I), 1{edge(I,C)}1.
leaf(I) :- node(I), 0{edge(I,C)}0.
      
multiplier(mul13;mul7;mul5).
        
bin_op(xor;and;mod;mul;sub).
% bin_op(xor;and).
un_op(mul13;mul7;mul5;neg).
null_op(const;key).

type(T) :- bin_op(T).
type(T) :- un_op(T).
type(T) :- null_op(T).
    
%%%Choose one operation for each node

1{op(I, T):bin_op(T)}1 :- binary(I).
1{op(I, T):un_op(T)}1 :- unary(I).
1{op(I, T):null_op(T)}1 :- leaf(I).
    

%%%Create constants
1 {const_bit(I,B,0..1)} 1 :- op(I,const),bit_index(B).

%%%Evaluate the dag, (Key, Id, Bit, Value)

%%% Basic operations (and,xor,neg,const,key)

val_bit(K,I,B,V1&V2) :- 
    op(I,and),
    edge(I,I1),
    edge(I,I2),
    I1 < I2, 
    val_bit(K,I1,B,V1), 
    val_bit(K,I2,B,V2).
    
val_bit(K,I,B,V1^V2) :- 
    op(I,xor),
    edge(I,I1),
    edge(I,I2),
    I1 < I2, 
    val_bit(K,I1,B,V1), 
    val_bit(K,I2,B,V2).
    
val_bit(K,I,B,1-V1) :- 
    op(I,neg),
    edge(I,I1),
    val_bit(K,I1,B,V1).
    
val_bit(K,I,B,V) :- 
    key(K), 
    op(I,const), 
    const_bit(I,B,V).
    
val_bit(K,I,B,V) :- 
    key_bit(K,B,V), 
    op(I,key).
    
%%% Advanced operations (mul,shl,shr,mod,div,add,sub), need yosys circuits
    
% if there is a node multiplying by 13, set the appropriate bit as input
input_signal(I,K,P,B,V1) :- 
    op(I,D),
    edge(I,I1),
    val_bit(K,I1,B,V1),
    device_port_direction(D,P,input,C).

2{input_signal(I,K,P1,B,V1);input_signal(I,K,P2,B,V2)}2 :-
    op(I,D),
    edge(I,I1),
    edge(I,I2),
    I1 < I2,
    %left_input(P1),
    %right_input(P2),
    device_port_direction(D,P1,input,C1),
    device_port_direction(D,P2,input,C2),
    C1 < C2,
    val_bit(K,I1,B,V1),
    val_bit(K,I2,B,V2).
    
val_bit(K,I,B,V) :- output_signal(I,K,B,V).

% get hash bit from node 0
hash_bit(K,B,V) :- val_bit(K,0,B,V).

% add up the hash bits for each key
hash_val(K,H) :- key(K), H = #sum{V*2**B:hash_bit(K,B,V)}.
    
:- hash_val(K,H), H > max_index.
    
%occupied(B,V) :- hash(K,B,V).
%collision :- occupied(V), 2 {hash(K,V)}.
    
%%%collision :- key(K1), key(K2), K1!=K2, hash(K1,V), hash(K2,V).
    
%%% Maybe we can use a custom theory progagator to ignore quagratic cost here
collision(K1,K2) :- 
    key(K1), 
    key(K2), 
    K1 < K2, 
    0 = #sum {V*2**B,v1:hash_bit(K1,B,V); -V*2**B,v2:hash_bit(K2,B,V)}.
    
:- collision(K1,K2).
    
%#minimize{V:node(V)}.
    
%%% Visualization rules

%#show children(I,N) : node(I), N = #count{C:edge(I,C)}.
%#show op_count(I, X) : node(I), X = #count{T:op(I,T),type(T)}.
    
%#show hash_bit/3.
%#show key/1.
#show edge/2.
#show op/2.
#show hash_val/2.


Overwriting ast.lp


I guess the above would make the basic structure of the AST, but still need some way assign values to the leaf nodes and allow those to be evaluated. No obivous way how to do that.

In [ ]:
!clingo ast.lp keys.lp circuits.lp multiplier.lp

clingo version 5.2.1
Reading from ast.lp ...


In [101]:
%%file keys.lp

key(0..10).
key(56).
key(342).
key(33).
key(124).
%key(209..251).

#script (python)
def bit_values(k,n):
    k = k.number
    n = n.number
    results = []
    for i in range(n):
        b = i
        v = 1 if (1<<i)&k else 0
        results.append((b,v))
    return results
#end.

key_bit(I,B,V) :- key(I), (B,V)=@bit_values(I,bits).

Overwriting keys.lp


In [69]:
!clingo keys.lp

clingo version 5.2.1
Reading from keys.lp
*** ERROR: (clingo): keys.lp:21:33-52: error: error calling python function:
  Traceback (most recent call last):
    File "<keys.lp:9:1-19:6>", line 6, in bit_values
  TypeError: range() integer end argument expected, got NoneType.

UNKNOWN

Models       : 0+
Calls        : 1
Time         : 0.037s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.025s


# Everything below is garbage

In [9]:
!clingo items.lp perfect.lp

clingo version 5.2.1
Reading from items.lp ...
Solving...
Answer: 1
item(123) item(23) item(55) item(67) item(100) item(34234) item(23123) item(1) item(2) item(3) item(4) item(5) item(6) item(7) item(8) item(9) item(10) coef(79) mod(83)
SATISFIABLE

Models       : 1+
Calls        : 1
Time         : 0.842s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.714s


In [10]:
%%file genhash.lp

#const corange = 10.
constant(1..corange).
input(1..5).


idrange(0..10).



%%% piece(id,child_id_1,child_id_2).
2 {piece(A,B,C):idrange(A),idrange(B),idrange(C)}.

%%% At most one piece per id
:- idrange(A), not 0 #sum{1,B,C:piece(A,B,C)} 1.
        
%%% Piece appears at most once as a child
:- idrange(P), not 0 #sum{1,A,C,left:piece(A,P,C); 1,A,B,right:piece(A,B,P)} 1.
    
%%% Piece can not be its own child
:- idrange(P), piece(P,P,_).
:- idrange(P), piece(P,_,P).
    
%%% If a piece appears as a child, it exists
%TODO
    

    
%:- idrange(B), not 0 #sum{1,A,C:piece(A,B,C)} 1.
%:- idrange(C), not 0 #sum{1,A,B:piece(A,B,C)} 1.

%cnt(X, A) :- idrange(A), X = #sum{1,B,C:piece(A,B,C)}.
    
    

Overwriting genhash.lp


In [11]:
%%file perfect.lp

%%% Attempts to perfectly hash keys given as item(number)

#const corange = 10.
constant(1..corange).
    
%%%operation(type, item, in1, in2, output)
operation(xor, I, A, B, A^B) :- item(I), output(I,A), output(I,B).       
operation(and, I, A, B, A&B) :- item(I), output(I,A), output(I,B). 
operation(or , I, A, B, A?B) :- item(I), output(I,A), output(I,B). 
    

hash(I,X) :- item(I), input(I,V1), input(I,V2), operation(T,I,V1,V2,X).
    
    
output(I,I) :- item(I).
output(I,X) :- item(I), constant(X).
output(I,X) :- item(I), output(I,V1), output(I,V2), operation(T,I,V1,V2,X).
    
    
:- item(A), item(B), A != B, hash(A,X), hash(B,X). 
    
    
    
%%% input constant or item
input(I) :- item(I).
input(I) :- constant(I).
    
    
%:- item(A), item(B), operation(T,ID,) 
    
    
%%output(I,X) :- input(I), operation(T, )
    




Overwriting perfect.lp


In [12]:
%%file minimal.lp

%%% Minimize the range of hashed values
maxhash(X) :- coef(A), mod(M), exp(E), X = #max{ @axbmodc(A,I,E,M):item(I) }.
minhash(X) :- coef(A), mod(M), exp(E), X = #min{ @axbmodc(A,I,E,M):item(I) }.
diff(X) :- maxhash(A), minhash(B), X = A-B.
#minimize{ X:diff(X) }.

Overwriting minimal.lp


In [13]:
%%file ordered.lp

%%% Reject if items are out of order when hashed
:- item(I1), item(I2), I1 < I2, coef(A), mod(M),  A*I1\M > A*I2\M.
:- #true.

Overwriting ordered.lp


In [14]:
!clingo genhash.lp

clingo version 5.2.1
Reading from genhash.lp
Solving...
Answer: 1
constant(1) constant(2) constant(3) constant(4) constant(5) constant(6) constant(7) constant(8) constant(9) constant(10) input(1) input(2) input(3) input(4) input(5) idrange(0) idrange(1) idrange(2) idrange(3) idrange(4) idrange(5) idrange(6) idrange(7) idrange(8) idrange(9) idrange(10) piece(0,1,2) piece(2,0,3)
SATISFIABLE

Models       : 1+
Calls        : 1
Time         : 0.032s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.031s


In [15]:
%%file process.py

import json
import sys
import re


result = json.load(sys.stdin)
atoms = result['Call'][0]['Witnesses'][0]['Value']

items = []
modulus = 0
coef = 0
exponent = 0

for a in atoms:
    match = re.match('item\(([0-9]*)\)', a)
    if match:
        items += [int(match.group(1))]
    
    match = re.match('mod\(([0-9]*)\)', a)
    if match:
        modulus = int(match.group(1))

    match = re.match('exp\(([0-9]*)\)', a)
    if match:
        exponent = int(match.group(1))
        
    match = re.match('coef\(([0-9]*)\)', a)
    if match:
        coef = int(match.group(1))

        
table = [(item*coef**exponent%modulus, item) for item in items]
table.sort()

print "Hash function: %dx^%d mod %d" %(coef,exponent,modulus)

print "%7s %7s" % ("hash", "item")
for item in table:
    print "%7d %7d" % item

Overwriting process.py


In [16]:
!clingo items.lp perfect.lp minimal.lp -t 8 --outf=2 --quiet=1 2>/dev/null | python2 process.py

Traceback (most recent call last):
  File "process.py", line 33, in <module>
    table = [(item*coef**exponent%modulus, item) for item in items]
ZeroDivisionError: integer division or modulo by zero


In [17]:
%%file gen.py
import random

seen = {}
i = 0
while i < 5:
    x = random.randint(10,100)
    if x not in seen:
        print "item(%d)." % (x)
        seen[x] = 1
        i+=1

Overwriting gen.py


In [18]:
!python2 gen.py > gen.lp; clingo gen.lp perfect.lp minimal.lp ordered.lp quiet.lp --outf=2 --quiet=1 2>/dev/null | python2 process.py

Traceback (most recent call last):
  File "process.py", line 8, in <module>
    atoms = result['Call'][0]['Witnesses'][0]['Value']
KeyError: 'Witnesses'


In [19]:
!python2 gen.py > gen.lp; clingo gen.lp perfect.lp minimal.lp -t 8 --outf=2 --quiet=1 2>/dev/null | python2 process.py

Traceback (most recent call last):
  File "process.py", line 33, in <module>
    table = [(item*coef**exponent%modulus, item) for item in items]
ZeroDivisionError: integer division or modulo by zero
